# Weak-supervised learning models for relation extraction

__Goal:__ Train weak-supervised models, get accuracy level

__Method:__ Test of weak-supervised DL models:
1. Train DL models on CORD-19 dataset
2. Extract relations from papers that was used in test dataset
3. Convert relations to BEL format
4. Compare with relations from covid-19 dataset, calculate accuracy
5. Run error analysis


__Data:__ covid-19-kg dataset, [CORD-19 processed by CoronaWhy](https://console.cloud.google.com/storage/browser/coronawhy/NLPDatasets/)

__Tools:__ [PyTorch](https://pytorch.org/), [OpenNRE](https://github.com/thunlp/OpenNRE), [Snorkel](https://www.snorkel.org/) [PyBEL](https://github.com/pybel/pybel)

__Result:__ Trained weak-supervised models, accuracy of weak-supervised models

In [1]:
#!python3 --version
#!echo $PYTHONPATH
# Update PYTHONPATH, by setting <USERNAME> below.  This is to ensure access to OpenNRE frameworks and models
#!export PYTHONPATH=/home/<USERNAME>/local/lib/python:/home/<USERNAME>/OpenNRE:/usr/local/lib/python3.7/site-packages
!export PYTHONPATH=/home/lani_lichtenstein/local/lib/python:/home/lani_lichtenstein/OpenNRE:/usr/local/lib/python3.7/site-packages

In [2]:
#!pip install requests pybel pandas requests indra pybel[jupyter]

In [3]:
import requests
import pybel
import pandas as pd
import matplotlib.pyplot as plt

import re

import tqdm
from tqdm import tqdm # not sure why you need both

import os
import json
import numpy as np
import collections
import matplotlib.pyplot as plt
import seaborn as sns

#from indra.processors import bel
from indra.sources import bel
from indra.util import batch_iter
from indra.sources import indra_db_rest

### Helper functions
These are copied from other task-vt notebooks (Protein Co-Occurrence)

In [4]:
def load_files(dirname):
    filenames = os.listdir(dirname)
    raw_files = []

    for filename in tqdm(filenames):
        filename = dirname + filename
        file = json.load(open(filename, 'rb'))
        raw_files.append(file)
    
    return raw_files


def get_all_files(dirname):
    all_files = []
    
    filenames = os.listdir(dirname)

    for filename in tqdm(filenames):
        filename = dirname + filename
        file = json.load(open(filename, 'rb'))
        all_files.append(file)
    
    return all_files

def get_cat_vocab(cat):
    df_cat = df[cat]
    items = df_cat.dropna().tolist()

    vocab_list = []

    for element in items:
        item = element.split(",")
        for e in item:
            vocab_list.append(e)
    
    c = collections.Counter()

    for word in vocab_list:
        c[word] += 1
        
    result_dic = dict(c)
    
    return result_dic

#https://www.kaggle.com/rtatman/co-occurrence-matrix-plot-in-python
def df_co_occurrance(df, strain_group):
  strains_df = df.copy()  
  for i in strain_group:
        eval_match = df.SARS_COV.str.contains(i)
        strains_df[i] = eval_match
  return strains_df


## Train CORD-19 neural relation extraction model 

 ###  Step 1 - Load and Pre-Process CORD-19 Annnotated Data
This dataset is taken from: https://github.com/SciBiteLabs/CORD19.

In [5]:
#load all files
# update rootpath to location of CORD19 data set
root_path='/mount_disk/CORD19/annotated-CORD-19/1.4/CORD19'
#path=os.path.join(root_path, 'benchmark/nyt10/nyt10_rel2id.json')))

dirs = [
    os.path.join(root_path, 'biorxiv_medrxiv/biorxiv_medrxiv/pdf_json/')
    #os.path.join(root_path, 'comm_use_subset/comm_use_subset/pdf_json/'),
    #os.path.join(root_path, 'custom_license/custom_license/pdf_json/'),
    #os.path.join(root_path, 'noncomm_use_subset/noncomm_use_subset/pdf_json/')
]

files_stack = []
for dir_ in dirs:
    files = get_all_files(dir_)
    files_stack.append(files)

100%|██████████| 1342/1342 [00:09<00:00, 137.16it/s]


In [6]:
#build list of entities types
# not sure if this blob of code is needed for neural relation extraction
c = collections.Counter()

cat_vocab = []

for files in tqdm(files_stack):
    for file in files:
        for block in file['body_text']:
            dict_file = block['termite_hits'].keys()
            for key in dict_file:
                cat_vocab.append(key)

for word in cat_vocab:
    c[word] += 1
   
vocab_list = (set(list(c.elements())))

100%|██████████| 1/1 [00:00<00:00, 29.57it/s]


In [7]:
#build dataframe: entity mentions by blocks ignoring hint count
features = []
for files in tqdm(files_stack):
    for file in files:
        paper_id = file['paper_id']
        
        i = 0
        sections = ['abstract', 'body_text']
        for section in sections:
            for block in file[section]:

                block_id = section + '_' + str(i)
                
                block_features = []
                block_features.append(paper_id)
                block_features.append(block_id)
                
                termite_hits = block['termite_hits']
                
                block_categories = termite_hits.keys()
                block_categories = list(block_categories)
                for cat in vocab_list:
        
                    if cat in block_categories:
                        cat_entities = []
                        for hit in termite_hits[cat]:
                            entity = hit.get('name')
                            if entity not in cat_entities:
                                cat_entities.append(entity)
                                
                        cat_entities = ",".join(cat_entities)

                    else:
                        cat_entities = None

                    block_features.append(cat_entities)

                features.append(block_features)
                i += 1


col_names = ['paper_id', 'block_id']
for cat in vocab_list:
    col_names.append(cat)
df = pd.DataFrame(features, columns=col_names)
df.head()

100%|██████████| 1/1 [00:00<00:00,  3.51it/s]


,paper_id,block_id,DRUG,INDICATION,CVPROT,COUNTRY,SPECIES,GENE,HPO,GOONTOL,SARSCOV
0,96ef1767754a53f792951ba1752440ae94e90c60,abstract_0,None,None,None,None,"Lymphocytic choriomeningitis virus,Viruses",None,None,None,None
1,96ef1767754a53f792951ba1752440ae94e90c60,body_text_1,None,None,None,None,Viruses,interferon gamma,None,"peptide binding,MHC protein binding,T cell rec...",None
2,50a217a2dacfe1364383ec8c681f64f2fd76dbe7,abstract_0,None,Coronavirus Infections,None,"Korea, Republic of,Italy",None,None,None,None,Severe acute respiratory syndrome coronavirus ...
3,50a217a2dacfe1364383ec8c681f64f2fd76dbe7,abstract_1,None,Coronavirus Infections,None,None,None,None,None,None,Severe acute respiratory syndrome coronavirus ...
4,50a217a2dacfe1364383ec8c681f64f2fd76dbe7,abstract_2,None,Coronavirus Infections,None,"Korea, Republic of,China,Italy",None,None,None,None,Severe acute respiratory syndrome coronavirus 2


#### Explore data that will be used for labelling

In [8]:
file['body_text'][0]['text'] # ok - so each text is already in a block

'WHO has announced on March 11 in Switzerland that SARS-CoV-2 has the characteristics of a global pandemic 1 . As of March 15, 2020, China has accumulated 81,048 cases of SARS-CoV-2 infection and a mortality rate of 3.95 %, also, there is a total of 64322 cases expect China, with a mortality rate of 3.51% 1,2 . The proportion of asymptomatic patients has gradually increased 3 . Asymptomatic patients are still highly contagious, paying special attention to the epidemiological characteristics of asymptomatic patients is of great significance for the control of the epidemic. '

### Step 2 - Load Covid19kg - manually annotated kg to get labelling data
This is our knowledge base, which we use for weak supervision 

In [9]:
#load graph pre-procesed by Charlie Hoyt: https://github.com/CoronaWhy/bel4corona/tree/master/data/covid19kg
url = 'https://github.com/CoronaWhy/bel4corona/raw/master/data/covid19kg/covid19-fraunhofer-grounded.bel.nodelink.json'
res = requests.get(url)
pybel_graph = pybel.from_nodelink(res.json())

# view graph in jupyter (not displaying)
#jupyter.to_html(pybel_graph)





In [12]:
#!pip install pybel-tools

In [10]:
#x=pybel_graph.edges.data()
import pybel_tools
from pybel_tools import summary


In [12]:
# returns a dict of key_val pairs 
edges=pybel_tools.summary.get_edge_relations(pybel_graph)


In [17]:
for key,value in edges.items():
    print(key)
    print(value)

(<BEL a(chebi:10100 ! zafirlukast)>, <BEL complex(a(chebi:10100 ! zafirlukast), p(hgnc:19679 ! AAK1))>)
{'partOf'}
(<BEL a(chebi:101278 ! diltiazem)>, <BEL a(chebi:38215 ! "calcium channel blocker")>)
{'isA'}
(<BEL a(chebi:101278 ! diltiazem)>, <BEL a(chebi:35674 ! "antihypertensive agent")>)
{'isA'}
(<BEL a(chebi:101278 ! diltiazem)>, <BEL path(mesh:D000787 ! "Angina Pectoris")>)
{'decreases'}
(<BEL a(chebi:101278 ! diltiazem)>, <BEL path(mesh:D001145 ! "Arrhythmias, Cardiac")>)
{'decreases'}
(<BEL a(chebi:101278 ! diltiazem)>, <BEL p(hgnc:5434 ! IFNB1)>)
{'increases'}
(<BEL a(chebi:101278 ! diltiazem)>, <BEL p(hgnc:18363 ! IFNL1)>)
{'increases'}
(<BEL a(chebi:101278 ! diltiazem)>, <BEL p(hgnc:18364 ! IFNL2)>)
{'increases'}
(<BEL a(chebi:101278 ! diltiazem)>, <BEL p(hgnc:18365 ! IFNL3)>)
{'increases'}
(<BEL a(chebi:101278 ! diltiazem)>, <BEL p(hgnc:44480 ! IFNL4)>)
{'increases'}
(<BEL a(chebi:101381 ! aloxistatin)>, <BEL p(hgnc:2527 ! CTSB)>)
{'decreases'}
(<BEL a(chebi:101381 ! aloxi

{'increases'}
(<BEL a(mesh:D016023 ! Integrins)>, <BEL bp(go:0007267 ! "cell-cell signaling")>)
{'increases'}
(<BEL a(mesh:D016207 ! Cytokines)>, <BEL path(doid:0080600 ! "COVID-19")>)
{'positiveCorrelation'}
(<BEL a(mesh:D016207 ! Cytokines)>, <BEL bp(go:0006954 ! "inflammatory response")>)
{'positiveCorrelation'}
(<BEL a(mesh:D016207 ! Cytokines)>, <BEL a(mesh:D015496 ! "CD4-Positive T-Lymphocytes")>)
{'decreases'}
(<BEL a(mesh:D016207 ! Cytokines)>, <BEL a(mesh:D018414 ! "CD8-Positive T-Lymphocytes")>)
{'decreases'}
(<BEL a(mesh:D016207 ! Cytokines)>, <BEL a(mesh:D050378 ! "T-Lymphocytes, Regulatory")>)
{'decreases'}
(<BEL a(mesh:D016676 ! "Macrophages, Alveolar")>, <BEL path(doid:10533 ! "viral pneumonia")>)
{'association'}
(<BEL a(mesh:D016756 ! "Immunoglobulins, Intravenous")>, <BEL path(doid:0080600 ! "COVID-19")>)
{'decreases'}
(<BEL a(mesh:D016898 ! "Interferon-alpha")>, <BEL path(mesh:D005221 ! Fatigue)>)
{'decreases'}
(<BEL a(mesh:D016898 ! "Interferon-alpha")>, <BEL path(me

{'positiveCorrelation'}
(<BEL bp(go:0072524 ! "pyridine-containing compound metabolic process")>, <BEL path(doid:0080600 ! "COVID-19")>)
{'negativeCorrelation', 'positiveCorrelation'}
(<BEL bp(go:0072525 ! "pyridine-containing compound biosynthetic process")>, <BEL path(doid:0080600 ! "COVID-19")>)
{'negativeCorrelation'}
(<BEL bp(go:0072583 ! "clathrin-dependent endocytosis")>, <BEL a(ncbitaxon:230237 ! "Porcine hemagglutinating encephalomyelitis virus (strain 67N)")>)
{'association'}
(<BEL bp(go:0072599 ! "establishment of protein localization to endoplasmic reticulum")>, <BEL path(doid:0080600 ! "COVID-19")>)
{'positiveCorrelation'}
(<BEL bp(go:0072641 ! "type I interferon secretion")>, <BEL bp(go:0009451 ! "RNA modification")>)
{'association'}
(<BEL bp(go:0075502 ! "endosome membrane permeabilization involved in viral entry into host cell")>, <BEL bp(go:0019079 ! "viral genome replication")>)
{'increases'}
(<BEL bp(go:0075502 ! "endosome membrane permeabilization involved in viral 

(<BEL p(hgnc:7200 ! MOV10)>, <BEL complex(p(hgnc:11617 ! ELOC), p(hgnc:7200 ! MOV10))>)
{'partOf'}
(<BEL p(hgnc:7200 ! MOV10)>, <BEL complex(p(hgnc:11623 ! TCF12), p(hgnc:7200 ! MOV10))>)
{'partOf'}
(<BEL p(hgnc:7200 ! MOV10)>, <BEL complex(p(hgnc:12611 ! USP13), p(hgnc:7200 ! MOV10))>)
{'partOf'}
(<BEL p(hgnc:7200 ! MOV10)>, <BEL complex(p(hgnc:12741 ! EIF4H), p(hgnc:7200 ! MOV10))>)
{'partOf'}
(<BEL p(hgnc:7200 ! MOV10)>, <BEL complex(p(hgnc:13631 ! RPL36), p(hgnc:7200 ! MOV10))>)
{'partOf'}
(<BEL p(hgnc:7200 ! MOV10)>, <BEL complex(p(hgnc:13722 ! NUTF2), p(hgnc:7200 ! MOV10))>)
{'partOf'}
(<BEL p(hgnc:7200 ! MOV10)>, <BEL complex(p(hgnc:14498 ! MRPS5), p(hgnc:7200 ! MOV10))>)
{'partOf'}
(<BEL p(hgnc:7200 ! MOV10)>, <BEL complex(p(hgnc:14511 ! MRPS25), p(hgnc:7200 ! MOV10))>)
{'partOf'}
(<BEL p(hgnc:7200 ! MOV10)>, <BEL complex(p(hgnc:14512 ! MRPS27), p(hgnc:7200 ! MOV10))>)
{'partOf'}
(<BEL p(hgnc:7200 ! MOV10)>, <BEL complex(p(hgnc:14559 ! STOML2), p(hgnc:7200 ! MOV10))>)
{'partOf'

(<BEL p(hgnc:14937 ! PIGS)>, <BEL complex(p(hgnc:14937 ! PIGS), p(hgnc:4446 ! GPAA1))>)
{'partOf'}
(<BEL p(hgnc:4446 ! GPAA1)>, <BEL complex(p(hgnc:14937 ! PIGS), p(hgnc:4446 ! GPAA1))>)
{'partOf'}
(<BEL p(hgnc:4446 ! GPAA1)>, <BEL complex(p(hgnc:3383 ! STOM), p(hgnc:4446 ! GPAA1))>)
{'partOf'}
(<BEL p(hgnc:4446 ! GPAA1)>, <BEL complex(p(hgnc:4446 ! GPAA1), p(hgnc:659 ! ARF6))>)
{'partOf'}
(<BEL p(hgnc:14988 ! POFUT1)>, <BEL complex(p(hgnc:14988 ! POFUT1), p(hgnc:7200 ! MOV10))>)
{'partOf'}
(<BEL p(hgnc:15563 ! IL37)>, <BEL complex(p(hgnc:15563 ! IL37), p(hgnc:5988 ! IL18R1))>)
{'partOf'}
(<BEL p(hgnc:15563 ! IL37)>, <BEL p(hgnc:9376 ! PRKAA1)>)
{'increases'}
(<BEL p(hgnc:15563 ! IL37)>, <BEL a(mesh:D000949 ! "Histocompatibility Antigens Class II")>)
{'decreases'}
(<BEL p(hgnc:15563 ! IL37)>, <BEL p(hgnc:7562 ! MYD88)>)
{'decreases'}
(<BEL p(hgnc:15563 ! IL37)>, <BEL p(hgnc:5992 ! IL1B)>)
{'decreases'}
(<BEL p(hgnc:15563 ! IL37)>, <BEL p(hgnc:6018 ! IL6)>)
{'decreases'}
(<BEL p(hgnc:15

{'partOf'}
(<BEL p(uniprot:P0C6X7 ! "R1AB_CVHSA", frag("5903_6429"))>, <BEL bp(go:0016032 ! "viral process")>)
{'increases'}
(<BEL p(uniprot:P0C6X7 ! "R1AB_CVHSA", frag("6430_6775"))>, <BEL composite(p(uniprot:P0C6X7 ! "R1AB_CVHSA", frag("5903_6429")), p(uniprot:P0C6X7 ! "R1AB_CVHSA", frag("6430_6775")))>)
{'partOf'}
(<BEL composite(p(uniprot:P0C6X7 ! "R1AB_CVHSA", frag("5903_6429")), p(uniprot:P0C6X7 ! "R1AB_CVHSA", frag("6776_7073")))>, <BEL bp(go:0019074 ! "viral RNA genome packaging")>)
{'increases'}
(<BEL p(uniprot:P0C6X7 ! "R1AB_CVHSA", frag("6776_7073"))>, <BEL composite(p(uniprot:P0C6X7 ! "R1AB_CVHSA", frag("5903_6429")), p(uniprot:P0C6X7 ! "R1AB_CVHSA", frag("6776_7073")))>)
{'partOf'}
(<BEL g(dbsnp:rs12252 ! rs12252)>, <BEL p(hgnc:5414 ! IFITM3)>)
{'association'}
(<BEL g(dbsnp:rs12252 ! rs12252)>, <BEL path(doid:0080600 ! "COVID-19")>)
{'positiveCorrelation'}
(<BEL g(hgnc.genefamily:1455 ! "Ferritin subunits")>, <BEL path(doid:0080600 ! "COVID-19")>)
{'positiveCorrelation'}
(

(<BEL path(doid:0080600 ! "COVID-19")>, <BEL path(hp:0003326 ! Myalgia)>)
{'association'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL path(hp:0002315 ! Headache)>)
{'association'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL path(doid:409 ! "liver disease")>)
{'increases', 'association'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL path(doid:863 ! "nervous system disease")>)
{'association'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL path(hp:0006528 ! "Chronic lung disease")>)
{'association'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL path(doid:784 ! "chronic kidney disease")>)
{'positiveCorrelation', 'association'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL path(doid:28 ! "endocrine system disease")>)
{'association'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL path(hp:0002664 ! Neoplasm)>)
{'association'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL bp(mesh:D018655 ! "Lymphocyte Count")>)
{'negativeCorrelation'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL path(hp:0011897 ! Neu

{'negativeCorrelation'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL r(hgnc:19173 ! CMTM2)>)
{'negativeCorrelation'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL r(hgnc:6025 ! CXCL8)>)
{'negativeCorrelation'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL r(hgnc:5438 ! IFNG)>)
{'negativeCorrelation'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL r(hgnc:2537 ! CTSL)>)
{'positiveCorrelation'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL r(hgnc:2527 ! CTSB)>)
{'positiveCorrelation'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL r(hgnc:24944 ! DDIT4)>)
{'positiveCorrelation'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL r(hgnc:10447 ! RRAS)>)
{'positiveCorrelation'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL r(hgnc:2529 ! CTSD)>)
{'positiveCorrelation'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL r(hgnc:593 ! BIRC5)>)
{'positiveCorrelation'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL r(hgnc:2547 ! CTSZ)>)
{'positiveCorrelation'}
(<BEL path(doid:0080600 ! "COVID-19")>, <BEL r(hgnc:8031 ! N

In [10]:

#export  graph as a tab-separated edge list.
#pybel.to_csv(pybel_graph, 'covid19kg_dataset.csv')


In [13]:
#pybel_pd=pd.read_csv('covid19kg_dataset.csv')

In [25]:
# Process the PyBEL graph into INDRA Statements
pybel_proc = bel.process_pybel_graph(pybel_graph)

# Note that of the ~4k statements in the PyBEL graph, only 831 are successfully
# converted to INDRA statements in large part because of issues with namespaces
covid_stmts = pybel_proc.statements
stmt_hashes = [s.get_hash() for s in covid_stmts]

# Explore Indra Statement
#len(covid_stmts) # 442
#type(covid_stmts[0]) # indra.statements.statements.Inhibition

#(covid_stmts[0])

INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace DOID with name influenza and identifier 8469 (path(doid:8469 ! influenza)).
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:8469 ! influenza)
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace DOID with name COVID-19 and identifier 0080600 (path(doid:0080600 ! "COVID-19")).
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:0080600 ! "COVID-19")
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace DOID with name COVID-19 and identifier 0080600 (path(doid:0080600 ! "COVID-19")).
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:0080600 ! "COVID-19")
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace DOID with name COVID-19 and identifier 00

INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace DOID with name COVID-19 and identifier 0080600 (path(doid:0080600 ! "COVID-19")).
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:0080600 ! "COVID-19")
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace DOID with name adult respiratory distress syndrome and identifier 11394 (path(doid:11394 ! "adult respiratory distress syndrome")).
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:11394 ! "adult respiratory distress syndrome")
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace HP with name Acute kidney injury and identifier 0001919 (path(hp:0001919 ! "Acute kidney injury")).
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: path(hp:0001919 ! "Acute kidney injury")
INFO: [

INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace DOID with name COVID-19 and identifier 0080600 (path(doid:0080600 ! "COVID-19")).
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:0080600 ! "COVID-19")
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace DOID with name COVID-19 and identifier 0080600 (path(doid:0080600 ! "COVID-19")).
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:0080600 ! "COVID-19")
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace DOID with name COVID-19 and identifier 0080600 (path(doid:0080600 ! "COVID-19")).
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:0080600 ! "COVID-19")
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace DOID with name COVID-19 and ide

INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:4449 ! "macular retinal edema")
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace DOID with name cardiomyopathy and identifier 0050700 (path(doid:0050700 ! cardiomyopathy)).
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:0050700 ! cardiomyopathy)
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace GO with name viral entry into host cell and identifier 0046718 (bp(go:0046718 ! "viral entry into host cell")).
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: bp(go:0046718 ! "viral entry into host cell")
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace GO with name regulation of pH and identifier 0006885 (bp(go:0006885 ! "regulation of pH")).
INFO: [2020-06-09 11:18:43] ind

INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace UNIPROT with name CP2C8_HUMAN and identifier P10632 (p(uniprot:P10632 ! "CP2C8_HUMAN")).
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: p(uniprot:P10632 ! "CP2C8_HUMAN")
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace UNIPROT with name CP2C8_HUMAN and identifier P10632 (p(uniprot:P10632 ! "CP2C8_HUMAN")).
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: p(uniprot:P10632 ! "CP2C8_HUMAN")
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace UNIPROT with name CP2C8_HUMAN and identifier P10632 (p(uniprot:P10632 ! "CP2C8_HUMAN")).
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unable to get identifier information for node: p(uniprot:P10632 ! "CP2C8_HUMAN")
INFO: [2020-06-09 11:18:43] indra.sources.bel.processor - Unhandled namespace UNIP

...

INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace GO with name neutrophil extravasation and identifier 0072672 (bp(go:0072672 ! "neutrophil extravasation")).
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: bp(go:0072672 ! "neutrophil extravasation")
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace DOID with name COVID-19 and identifier 0080600 (path(doid:0080600 ! "COVID-19")).
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:0080600 ! "COVID-19")
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace DOID with name COVID-19 and identifier 0080600 (path(doid:0080600 ! "COVID-19")).
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:0080600 ! "COVID-19")
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled nam

INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: bp(go:0043542 ! "endothelial cell migration")
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace DOID with name atherosclerosis and identifier 1936 (path(doid:1936 ! atherosclerosis)).
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:1936 ! atherosclerosis)
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace GO with name vascular associated smooth muscle cell migration and identifier 1904738 (bp(go:1904738 ! "vascular associated smooth muscle cell migration")).
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: bp(go:1904738 ! "vascular associated smooth muscle cell migration")
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace DOID with name severe acute respiratory syndrome and identi

INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace GO with name macrophage activation and identifier 0042116 (bp(go:0042116 ! "macrophage activation")).
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: bp(go:0042116 ! "macrophage activation")
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace DOID with name severe acute respiratory syndrome and identifier 2945 (path(doid:2945 ! "severe acute respiratory syndrome")).
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:2945 ! "severe acute respiratory syndrome")
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace DOID with name severe acute respiratory syndrome and identifier 2945 (path(doid:2945 ! "severe acute respiratory syndrome")).
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: pa

INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: bp(go:0000045 ! "autophagosome assembly")
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace HP with name Hyperactive renin-angiotensin system and identifier 0000841 (path(hp:0000841 ! "Hyperactive renin-angiotensin system")).
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: path(hp:0000841 ! "Hyperactive renin-angiotensin system")
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace DOID with name hypertension and identifier 10763 (path(doid:10763 ! hypertension)).
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:10763 ! hypertension)
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace HP with name Hyperactive renin-angiotensin system and identifier 0000841 (path(hp:0000841 ! "Hyperact

INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: complex(go:0005783 ! "endoplasmic reticulum")
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace EFO with name Endoplasmic Reticulum Stress and identifier 1001886 (path(efo:1001886 ! "Endoplasmic Reticulum Stress")).
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: path(efo:1001886 ! "Endoplasmic Reticulum Stress")
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace DOID with name adult respiratory distress syndrome and identifier 11394 (path(doid:11394 ! "adult respiratory distress syndrome")).
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unable to get identifier information for node: path(doid:11394 ! "adult respiratory distress syndrome")
INFO: [2020-06-09 11:18:45] indra.sources.bel.processor - Unhandled namespace DOID with name adult respiratory distress syn

### Step3 - Label CORD-19 data with snorkel for Weak Supervision
Create label functions here to label data heuristically

Extract a sentence from CORD-19  
“'The SARS-CoV-2 PTX targets GENE HCG'”

First extract candidate entity pairs (e1,e2) from each sentence using a trained NER (e.g. spacy with scispacy?)

Then, does a relationship (e1, 'partOF' ,e2) exist in knowledge base (covid19kg?)

If yes, label sentence with relation “partOf”

Add entity pair with lable RX to candidate set

For each relation, keep 1000 instances with unique entity pairs



In [24]:
import snorkel

In [26]:
# Define the label mappings for convenience
#ABSTAIN = -1
#NOT_SPAM = 0
#SPAM = 1

# Labels are relations - perhaps all BEL relations?

relations_pybel=pybel.struct.summary.count_relations(pybel_graph)

# turn existing relations in pybel into a list
relations=[]
for i in relations_pybel.keys():
    relations.append(i)

In [28]:
# print out relations in the Knowledge Base
#relations

In [29]:
# Define labeling function.. 
from snorkel.labeling import labeling_function

@labeling_function()
def lf_keyword_my(x):
    """Many spam comments talk about 'my channel', 'my video', etc."""
    return SPAM if "my" in x.text.lower() else ABSTAIN

### Step 4 - Train a Classifer with openNRE


Support set S, has labelled instances (x1, r1), (x2, r2)
 x1 is a sentence embedding (feature vector) generated through e.g. CNN or say, BERT 

Use meta-learning (example of few-shot learner) to update parameters of the classifier. 


For training
- need to have train,test, validate data (see examples in openNRE/pretrain/wiki80 for format)
- need to use a LM for encoding e.g. BERT of glove 
- need to select Meta Learner model (e.g. bagged attention)

In [30]:
import opennre

INFO: [2020-06-10 09:55:13] transformers.file_utils - PyTorch version 1.3.0 available.


In [11]:
# Example - check example works
model = opennre.get_model('wiki80_cnn_softmax')
model.infer({'text': 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).', 'h': {'pos': (18, 46)}, 't': {'pos': (78, 91)}})

('father', 0.7500484585762024)

In [31]:
# Exploration of OpenNRE
# Explore wiki80 training data (to get an idea of training data format)
# Load the file
path='/home/lani_lichtenstein/.opennre/benchmark/wiki80/wiki80_train.txt'
f = open(path)
data = []
for line in f.readlines():
    line = line.rstrip()
    if len(line) > 0:
        data.append(eval(line))
      

In [32]:
# explore data
type(data[0]) # data is a list of dicts 
data[0]


{'token': ['Merpati',
  'flight',
  '106',
  'departed',
  'Jakarta',
  '(',
  'CGK',
  ')',
  'on',
  'a',
  'domestic',
  'flight',
  'to',
  'Tanjung',
  'Pandan',
  '(',
  'TJQ',
  ')',
  '.'],
 'h': {'name': 'tjq', 'id': 'Q1331049', 'pos': [16, 17]},
 't': {'name': 'tanjung pandan', 'id': 'Q3056359', 'pos': [13, 15]},
 'relation': 'place served by transport hub'}

In [ ]:
data[0]['t'] # explore tail value of first element


In [14]:
wiki_data=pd.DataFrame(data)
wiki_data.columns
wiki_data.shape
wiki_data.iloc[0]
wiki_data.head()

,token,h,t,relation
0,"[Merpati, flight, 106, departed, Jakarta, (, C...","{'name': 'tjq', 'id': 'Q1331049', 'pos': [16, ...","{'name': 'tanjung pandan', 'id': 'Q3056359', '...",place served by transport hub
1,"[The, name, was, at, one, point, changed, to, ...","{'name': 'east midlands airport', 'id': 'Q8977...","{'name': 'nottingham', 'id': 'Q41262', 'pos': ...",place served by transport hub
2,"[It, is, a, four, -, level, stack, interchange...",{'name': 'fort lauderdale-hollywood internatio...,"{'name': 'fort lauderdale, florida', 'id': 'Q1...",place served by transport hub
3,"[It, is, the, main, alternate, of, Jinnah, Int...","{'name': 'jinnah international airport', 'id':...","{'name': 'karachi', 'id': 'Q8660', 'pos': [10,...",place served by transport hub
4,"[Nearby, airports, include, Akwa, Ibom, Airpor...",{'name': 'margaret ekpo international airport'...,"{'name': 'calabar', 'id': 'Q844091', 'pos': [1...",place served by transport hub


### Train - use Bert as model, on our training data, 

In [ ]:

#https://github.com/thunlp/OpenNRE/blob/master/example/train_nyt10_pcnn_att.py
    
import sys, json
import torch
import os
import numpy as np
import opennre
from opennre import encoder, model, framework
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--bag_size', type=int, default=0)
args = parser.parse_args()

# Some basic settings
root_path = '.'
if not os.path.exists('ckpt'):
    os.mkdir('ckpt')
ckpt = 'ckpt/nyt10_pcnn_att.pth.tar'

# Check data
opennre.download('nyt10', root_path=root_path)
opennre.download('glove', root_path=root_path)
rel2id = json.load(open(os.path.join(root_path, 'benchmark/nyt10/nyt10_rel2id.json')))
wordi2d = json.load(open(os.path.join(root_path, 'pretrain/glove/glove.6B.50d_word2id.json')))
word2vec = np.load(os.path.join(root_path, 'pretrain/glove/glove.6B.50d_mat.npy'))

# Define the sentence encoder
sentence_encoder = opennre.encoder.PCNNEncoder(
    token2id=wordi2d,
    max_length=120,
    word_size=50,
    position_size=5,
    hidden_size=230,
    blank_padding=True,
    kernel_size=3,
    padding_size=1,
    word2vec=word2vec,
    dropout=0.5
)

# Define the model
model = opennre.model.BagAttention(sentence_encoder, len(rel2id), rel2id)

# Define the whole training framework
framework = opennre.framework.BagRE(
    train_path='benchmark/nyt10/nyt10_train.txt',
    val_path='benchmark/nyt10/nyt10_val.txt',
    test_path='benchmark/nyt10/nyt10_test.txt',
    model=model,
    ckpt=ckpt,
    batch_size=160,
    max_epoch=60,
    lr=0.5,
    weight_decay=0,
    opt='sgd',
    bag_size=args.bag_size)

# Train the model
framework.train_model()

# Test the model
framework.load_state_dict(torch.load(ckpt)['state_dict'])
result = framework.eval_model(framework.test_loader)

# Print the result
print('AUC on test set: {}'.format(result['auc']))

### Compare with relations from covid19kg dataset

In [ ]:
#load graph pre-procesed by Charlie Hoyt: https://github.com/CoronaWhy/bel4corona/tree/master/data/covid19kg
url = 'https://github.com/CoronaWhy/bel4corona/raw/master/data/covid19kg/covid19-fraunhofer-grounded.bel.nodelink.json'
res = requests.get(url)